In [3]:
import pandas as pd
import json
import fasttext
import emoji
import re
import collections
from urllib.parse import urlparse

# Loading merged data from channels

In [ ]:
df_channels = pd.read_csv('/Users/katerynaburovova/PycharmProjects/dehumanization/data/merged_dataset/df_channels.csv', header=None)

In [3]:
df_channels.columns = df_channels.iloc[0]
df_channels = df_channels[1:]

In [4]:
df_channels = (df_channels
               .drop(columns=['Unnamed: 0'], axis=1)
               .reset_index(drop=True))

In [5]:
len(df_channels)

8108693

In [6]:
df_channels.sample(5)

,id,date,views,reactions,to_id,fwd_from,message,type,duration,channel_name,frw_from_title,frw_from_name,msg_entity
6180152,944.0,2018-01-18 06:05:49+00:00,788.0,NaN,PeerChannel(channel_id=1138542535),NaN,"103 года назад, 18 января 1915 года родился вы...",photo,NaN,regnum_na,NaN,NaN,NaN
1059643,102109.0,2022-01-20 06:26:23+00:00,6260.0,MessageReactions(results=[ReactionCount(reacti...,PeerChannel(channel_id=1050820672),NaN,Ростуризм впервые лишил права аккредитации орг...,text,NaN,tass_agency,NaN,NaN,NaN
6144226,2548.0,2022-06-23 16:24:40+00:00,50676.0,MessageReactions(results=[ReactionCount(reacti...,PeerChannel(channel_id=1644818949),NaN,Сегодня ночью в Барселоне будут отмечать празд...,video,9.0,berdovaalena,NaN,NaN,NaN
1106745,54596.0,2021-05-19 13:32:59+00:00,9731.0,NaN,PeerChannel(channel_id=1050820672),NaN,❗ Бывший вице-губернатор Санкт-Петербурга Оган...,text,NaN,tass_agency,NaN,NaN,NaN
332033,11191.0,2022-06-16 20:32:16+00:00,58287.0,NaN,PeerChannel(channel_id=1171552896),NaN,"Читаю материалы о том, что базу для энергетиче...",text,NaN,russ_orientalist,NaN,NaN,NaN


## We drop na for now (needs some mending)

In [7]:
df_channels = df_channels[~df_channels['message'].isna()]

In [8]:
f"We have {len(df_channels[(df_channels['message']=='nan')&(df_channels['type']=='text')].channel_name.unique())/len(df_channels):.4%} nans"

'We have 0.0000% nans'

### Loading list of unique channel name handles referenced by the group

In [9]:
names = (df_channels['frw_from_name']
              .value_counts()
              .reset_index(name="count")[1:]['index']
              .to_list())
len(names)

1959

In [10]:
len(names)

1959

In [11]:
dict_names = {"titles": names}
json_object = json.dumps(dict_names, indent=4)
with open("/Users/katerynaburovova/PycharmProjects/dehumanization/data/names_channels_list.json", "w") as outfile:
    outfile.write(json_object)

# Exploring the data

In [12]:
class LanguageIdentification:

    def __init__(self):
        pretrained_lang_model = "/Users/katerynaburovova/PycharmProjects/comp_soc_sci_projects/fasttext/lid.176.bin"
        self.model = fasttext.load_model(pretrained_lang_model)

    def predict_lang(self, text, label_only=True):
        predictions = self.model.predict(text, k=1)
        if label_only:
            return predictions[0][0][-2:]
        return predictions

In [13]:
lang_identifier = LanguageIdentification()
df_channels['message'] = df_channels['message'].apply(lambda x: str(x).replace('\n', ' '))
df_channels['lang'] = df_channels['message'].apply(lambda x: lang_identifier.predict_lang(x))

In [14]:
df_channels['lang'].value_counts()

ru    6901187
en      48321
uk      21941
bg       5814
de       5520
       ...   
mf          1
an          1
nb          1
iq          1
ep          1
Name: lang, Length: 138, dtype: int64

# Text preprocesing

In [15]:
df_channels.sample(10)

,id,date,views,reactions,to_id,fwd_from,message,type,duration,channel_name,frw_from_title,frw_from_name,msg_entity,lang
4482895,58373.0,2021-05-09 17:48:06+00:00,49581.0,NaN,PeerChannel(channel_id=1315735637),"MessageFwdHeader(date=datetime.datetime(2021, ...",✔️Полиция Украины возбудила как минимум три уг...,text,NaN,SolovievLive,NaN,NaN,NaN,ru
4331800,57594.0,2018-12-05 06:27:23+00:00,6946.0,NaN,PeerChannel(channel_id=1054549314),NaN,Как перевезти трубу на скутере и заставить все...,video,16.0,ntvnews,NaN,NaN,NaN,ru
2934195,24254.0,2021-11-05 16:01:18+00:00,31116.0,NaN,PeerChannel(channel_id=1038402501),NaN,🖼 Искусственный интеллект обнажил Пикассо Лон...,photo,NaN,kommersant,NaN,NaN,NaN,ru
2405853,87674.0,2022-01-08 17:36:39+00:00,41459.0,NaN,PeerChannel(channel_id=1036362176),NaN,В Пентагоне в преддверии переговоров с Россией...,text,NaN,rt_russian,NaN,NaN,NaN,ru
6823529,3543.0,2020-11-03 18:05:38+00:00,11083.0,NaN,PeerChannel(channel_id=1492765963),NaN,#MovsesGhazaryan для канала 🛑Карнаухов *** На...,text,NaN,sskarnaukhov,NaN,NaN,NaN,ru
2500248,39447.0,2022-05-10 15:45:01+00:00,220496.0,MessageReactions(results=[ReactionCount(reacti...,PeerChannel(channel_id=1100158992),NaN,"""Служу России!"" В Херсонской области наградил...",video,79.0,shot_shot,NaN,NaN,NaN,ru
6359965,33651.0,2022-04-06 12:08:56+00:00,888647.0,NaN,PeerChannel(channel_id=1117628569),NaN,Его либерально-демократическая партия появилас...,video,82.0,breakingmash,NaN,NaN,NaN,ru
332712,10367.0,2022-03-16 17:44:14+00:00,95233.0,MessageReactions(results=[ReactionCount(reacti...,PeerChannel(channel_id=1171552896),NaN,США передаст Украине 9 тыс. противотанковых ко...,text,NaN,russ_orientalist,NaN,NaN,NaN,ru
1929906,2859.0,2019-11-15 20:14:22+00:00,49905.0,NaN,PeerChannel(channel_id=1167445017),NaN,Сделку по С-400 с Индией многие связывали с те...,photo,NaN,politadequate,NaN,NaN,NaN,ru
7804942,162430.0,2019-07-29 07:18:25+00:00,2.0,NaN,PeerChannel(channel_id=1082084045),"MessageFwdHeader(date=datetime.datetime(2019, ...","По оценкам экспертов, в Германии примерно 12 т...",photo,NaN,karaulny,NaN,NaN,NaN,ru


In [16]:
test_dataset = ['test sentence 1', 'test sentence 2','test sentence 3']

In [17]:
with open('test_dataset.txt', 'w', encoding='utf8') as f:
    for line in test_dataset:
        f.write(f"{line}\n")

## Separating comments from posts

In [18]:
df_channels[df_channels['channel_name']=='Topaz_Govorit']

,id,date,views,reactions,to_id,fwd_from,message,type,duration,channel_name,frw_from_title,frw_from_name,msg_entity,lang
5847662,2762.0,2022-12-02 18:01:03+00:00,8538.0,MessageReactions(results=[ReactionCount(reacti...,PeerChannel(channel_id=1321128351),NaN,"Помнится, что за месяц до начала войны с Софий...",text,NaN,Topaz_Govorit,NaN,NaN,NaN,ru
5847663,980465.0,2022-12-02 18:01:55+00:00,NaN,NaN,PeerChannel(channel_id=1679205140),NaN,На бумаге никакой войны нет,text,NaN,Topaz_Govorit,NaN,NaN,NaN,ru
5847664,980466.0,2022-12-02 18:02:11+00:00,NaN,MessageReactions(results=[ReactionCount(reacti...,PeerChannel(channel_id=1679205140),NaN,Так это закономерность ВС РФ демонстрируют не ...,text,NaN,Topaz_Govorit,NaN,NaN,NaN,ru
5847665,980467.0,2022-12-02 18:02:31+00:00,NaN,NaN,PeerChannel(channel_id=1679205140),NaN,и при всём при этом униатская церковь всё ещё ...,text,NaN,Topaz_Govorit,NaN,NaN,NaN,ru
5847666,980468.0,2022-12-02 18:02:37+00:00,NaN,NaN,PeerChannel(channel_id=1679205140),NaN,Интересно что скажет патриарх.,text,NaN,Topaz_Govorit,NaN,NaN,NaN,ru
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5860896,10.0,2019-07-29 16:08:49+00:00,1793.0,MessageReactions(results=[ReactionCount(reacti...,PeerChannel(channel_id=1321128351),NaN,Наверное каждый на своём пути встречал такого ...,photo,NaN,Topaz_Govorit,NaN,NaN,NaN,ru
5860898,8.0,2019-07-29 15:31:10+00:00,1727.0,MessageReactions(results=[ReactionCount(reacti...,PeerChannel(channel_id=1321128351),NaN,А ещё иногда будут смешные для меня мемы собст...,text,NaN,Topaz_Govorit,NaN,NaN,NaN,ru
5860899,7.0,2019-07-29 15:15:42+00:00,1924.0,MessageReactions(results=[ReactionCount(reacti...,PeerChannel(channel_id=1321128351),NaN,"Если хотите изменить мир вокруг себя, не нужно...",text,NaN,Topaz_Govorit,NaN,NaN,NaN,ru
5860900,6.0,2019-07-29 14:20:06+00:00,1935.0,MessageReactions(results=[ReactionCount(reacti...,PeerChannel(channel_id=1321128351),NaN,Интересную вещь заметил: каждый раз когда в МС...,text,NaN,Topaz_Govorit,NaN,NaN,NaN,ru


In [19]:
id_list = df_channels[['channel_name', 'to_id']].drop_duplicates(keep='first').drop_duplicates(subset=['channel_name'], keep='first').to_id.tolist()

In [20]:
df_channels['is_post'] = df_channels.apply(lambda x: True if x.to_id in id_list else False, axis=1)

In [21]:
print(f'{len(df_channels[df_channels["is_post"]==True])/len(df_channels)*100:.1f}% of messages are posts, the rest are comments')

97.4% of messages are posts, the rest are comments


In [22]:
df_comments = df_channels[df_channels['is_post']==False]

In [23]:
df_posts = df_channels[df_channels['is_post']==True]

In [24]:
# df_posts.type = pd.api.types.CategoricalDtype(categories=df_posts.type.unique().tolist(), ordered = False)

In [25]:
# df_posts.type = pd.api.types.CategoricalDtype(categories=df_posts.type.unique().tolist(), ordered = False)


In [26]:
# df_posts.lang = pd.api.types.CategoricalDtype(categories=df_posts.lang.unique().tolist(), ordered = False)

In [27]:
# df_posts.to_csv('only_posts.csv', index = False, header=True)

In [28]:
df_channels = df_channels[df_channels['is_post']==True]

In [29]:
df_channels.channel_name.unique()

array(['mardanaka', 'rian_ru', 'tvrain', 'krispotupchik', 'akimapachev',
       'go338', 'KotNaMirotvorze', 'emphasises', 'russ_orientalist',
       'pravda_shuravi', 'voenacher', 'n_zackhaim', 'Hinshtein',
       'kashinguru', 'lentachold', 'wargonzo', 'foxandraven',
       'madam_secretar', 'russianfuture', 'vchkogpu', 'rusich_army',
       'botcharov', 'mashmoyka', 'vv_volodin', 'Mikle1On',
       'kremlinprachka', 'ctrs2018', 'mediazzzona', 'sotaproject',
       'znachit_net', 'er_molnia', 'razvedkavperedZ', 'rasstrelny',
       'govoritfursov', 'gramotyyaroslava', 'informnapalm',
       'podosokorsky', 'mosnow', 'daokedao', 'chtddd', 'nevzorovtv',
       'SonOfMonarchy', 'lesyaryabtseva', 'fontankaspb', 'tass_agency',
       'ErnestV_2020', 'sorok40russia', 'odinokayakoko', 'pgubarev',
       'swodki', 'meduzalive', 'Gori_spb', 'anna_news', 'readovkanews',
       'umar_kremlev', 'government_rus', 'Alekhin_Telega', 'leylinurimm',
       'pushilindenis', 'strelets_molodec', 'istrkal

## Removing breaks and repr symbols

In [30]:
test_string = df_channels[df_channels['is_post']==True].message[0]
test_string

'А трансляция тем временем идет. Подписывайтесь  https://www.youtube.com/watch?v=4L7T3u7utSw'

In [31]:
splitters = ['\n', '\t', '\\n','\xa0', '\u200b']

In [32]:
df_channels['message_no_breaks'] = df_channels['message'].str.replace('\n|\t|\\n', ' ', case=False)

/var/folders/2w/hygxgpp5353_dy_tk88gw2100000gn/T/ipykernel_82062/3913600352.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df_channels['message_no_breaks'] = df_channels['message'].str.replace('\n|\t|\\n', ' ', case=False)


## Isolating emojis

In [33]:
def get_emoji_count(text):
    return collections.Counter([match["emoji"] for word in text for match in emoji.emoji_list(word)])

In [34]:
def remove_emoji(string):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r' ', string)

In [35]:
%%time
df_channels['emojis'] = df_channels['message'].apply(lambda x: get_emoji_count(x))

CPU times: user 20min 26s, sys: 33.2 s, total: 20min 59s
Wall time: 24min 20s


In [36]:
%%time
df_channels['message_no_emoji'] = df_channels['message_no_breaks'].apply(lambda x: remove_emoji(x))

CPU times: user 46.4 s, sys: 28.6 s, total: 1min 15s
Wall time: 2min 48s


In [37]:
df_channels['message_no_emoji'].sample(10).iloc[0]

'Китай возмущен «беспределом» компании Илона Маска SpaceX в освоении космического пространства: в докладе, представленном Пекином Комитету ООН по использованию космического пространства в мирных целях в начале этого месяца, Китай заявил, что его космическая станция была вынуждена дважды – в июле и октябре 2021 года - менять орбитальные параметры  своей космической станции, чтобы избежать столкновения со спутниками Starlink.  SpaceX уже запустила в космос более 1900 спутников для создания орбитальной группировки космической телекоммуникационной сети Starlink. Федеральная комиссия по связи США выдала разрешение компании Илона Маска на запуск в космос до 12000 спутников.  Пекин призвал ООН напомнить США о международных договорах и соглашениях по использованию космического пространства и об ответственности государств за деятельность, осуществляемую как правительственными, так и неправительственными организациями в космосе.  Однако США не остались в долгу: представитель Гарвард-Смитсоновско

## Isolating urls

In [38]:
def find_urls(text):
    try:
        links =  re.findall(r'(https?://[^\s]+)', text)
        return [urlparse(item).netloc for item in links]
    except ValueError:
        return []

In [39]:
df_channels['url_list'] = df_channels['message_no_emoji'].apply(lambda x: find_urls(x))

In [40]:
df_channels['message_no_urls'] = df_channels['message_no_emoji'].str.replace('http\S+|www.\S+', '', case=False)

/var/folders/2w/hygxgpp5353_dy_tk88gw2100000gn/T/ipykernel_82062/343568952.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df_channels['message_no_urls'] = df_channels['message_no_emoji'].str.replace('http\S+|www.\S+', '', case=False)


In [41]:
#bs check
(df_channels['message_no_urls']
 .apply(lambda x: find_urls(x))
 .value_counts())

[]    6836544
Name: message_no_urls, dtype: int64

In [42]:
df_channels['message_no_urls'].sample(10).iloc[0]

'В казанском цирке слоны устроили драку во время представления.  На кадрах из соцсетей видно, как посетители разбегаются со зрительских мест. По предварительным данным, никто не пострадал.  На сайте цирка опубликовано сообщение, что представление было отменено «по техническим причинам»'

## Extract mentions

In [43]:
df_channels['mentions'] = df_channels['message_no_urls'].str.findall(r"@([a-zA-Z0-9_]{1,50})")

In [44]:
df_channels[['mentions','message_no_urls']].sample(10)
# df_channels[['mentions','message_no_urls']].message_no_urls.iloc[1265478]

,mentions,message_no_urls
155671,[],Террористическая атака на пакистанскую биржу....
5248504,[],"Утренняя подборка новостей, чтобы начать день ..."
3968835,[],«И весь мир в ядерный пепел»: Ким Чен Ын обсуд...
5100258,[],Изьятый у криминала украинского города Покровс...
6926988,[],Последние данные о распространении коронавирус...
1400945,[],По Лиману нанесено не менее пяти авиаударов. Р...
2916705,[kommersant],США ускорили развертывание на базах НАТО в Евр...
6816144,"[sputnikKZ, sputnikKZ]","Украинские сценаристы ""языковых рейдов"" в Каза..."
6879108,[],Названа дата похорон Сергея Соловьёва Погребе...
1122307,[],Заместитель министра иностранных дел России Се...


In [45]:
# text_tst = df_channels[['mentions','message_no_urls']].message_no_urls.iloc[1265478]
# m_list = df_channels[['mentions','message_no_urls']].mentions.iloc[1265478]

In [46]:
def cut_mentions(list_of_mentions, text):
    for word in list_of_mentions:
        wrd = '@' + word
        text = text.replace(wrd, '')
    return text.strip()

In [47]:
%%time
df_channels['message_no_mentions'] = df_channels.apply(lambda x: cut_mentions(x['mentions'], x['message_no_urls']), axis=1)

CPU times: user 36.6 s, sys: 4min 7s, total: 4min 44s
Wall time: 24min 54s


In [50]:
mentions_col_list = df_channels[df_channels['mentions'].map(lambda d: len(d)) > 0]['mentions'].tolist()
mentions_list = [item for sublist in mentions_col_list for item in sublist]
mentions_unique = list(set(mentions_list))

In [51]:
print(f'We have {len(mentions_unique)} unique channel names among {len(mentions_list)} mentions for dataset')

We have 24789 unique channel names among 1454907 mentions for dataset


In [52]:
dict = {"names": mentions_list}
json_object = json.dumps(dict, indent=4)
with open("/Users/katerynaburovova/PycharmProjects/dehumanization/data/names_from_mentions.json", "w") as outfile:
    outfile.write(json_object)

## Unify and isolate quotes

In [55]:
to_replace = ["«", "»", "“"]

df_channels['message_no_mentions'] = df_channels['message_no_mentions'].str.replace('|'.join(to_replace),'"')

/var/folders/2w/hygxgpp5353_dy_tk88gw2100000gn/T/ipykernel_82062/4136279409.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  df_channels['message_no_mentions'] = df_channels['message_no_mentions'].str.replace('|'.join(to_replace),'"')


In [56]:
%%time
df_channels['quotes'] = df_channels['message_no_mentions'].str.findall(r'"(.*?)"')

CPU times: user 9.43 s, sys: 41.8 s, total: 51.2 s
Wall time: 4min 5s


## Isolate hashtags

In [57]:
%%time
# df_channels['hashtags'] = df_channels['message_no_mentions'].str.findall("#[A-Za-z0-9_]+")
df_channels['hashtags'] = df_channels['message_no_mentions'].apply(lambda x: {word.strip(",").strip(".") for word in x.split() if word.startswith("#")})

CPU times: user 46.4 s, sys: 34.9 s, total: 1min 21s
Wall time: 4min 7s


In [59]:
def cut_hasgtags(set_of_hashtags, text):
    for word in list(set_of_hashtags):
        text = text.replace(word, '')
    return text.strip()

In [62]:
# df_channels.to_csv('/Users/katerynaburovova/PycharmProjects/dehumanization/datasets/df_channels_before_sym_cleanup.csv')

In [ ]:
df_channels['message_no_hashtags'] = df_channels.apply(lambda x: cut_hasgtags(x['hashtags'], x['message_no_mentions']), axis=1)

In [1]:
import pandas as pd
df_channels = pd.read_csv('/Users/katerynaburovova/PycharmProjects/dehumanization/datasets/df_channels_before_sym_cleanup.csv')

/var/folders/2w/hygxgpp5353_dy_tk88gw2100000gn/T/ipykernel_6652/229763949.py:2: DtypeWarning: Columns (4,6,11,12,13) have mixed types. Specify dtype option on import or set low_memory=False.
  df_channels = pd.read_csv('/Users/katerynaburovova/PycharmProjects/dehumanization/datasets/df_channels_before_sym_cleanup.csv')


In [3]:
df_channels.sample(5)

,Unnamed: 0,id,date,views,reactions,to_id,fwd_from,message,type,duration,...,message_no_breaks,emojis,message_no_emoji,url_list,message_no_urls,mentions,message_no_mentions,quotes,hashtags,message_no_hashtags
2848815,3437927,19002.0,2018-12-26 21:03:03+00:00,2299.0,NaN,PeerChannel(channel_id=1071040207),NaN,Ну что с нами сделали если даже название старо...,text,NaN,...,Ну что с нами сделали если даже название старо...,Counter(),Ну что с нами сделали если даже название старо...,[],Ну что с нами сделали если даже название старо...,[],Ну что с нами сделали если даже название старо...,[],set(),Ну что с нами сделали если даже название старо...
6086887,7184869,7339.0,2018-12-20 10:45:45+00:00,14655.0,NaN,PeerChannel(channel_id=1109403194),NaN,"Владимир Владимирович, мне кажется, не так пон...",text,NaN,...,"Владимир Владимирович, мне кажется, не так пон...",Counter(),"Владимир Владимирович, мне кажется, не так пон...",[],"Владимир Владимирович, мне кажется, не так пон...",[],"Владимир Владимирович, мне кажется, не так пон...",[],set(),"Владимир Владимирович, мне кажется, не так пон..."
1036036,1230819,10020.0,2021-05-25 19:33:54+00:00,6257.0,NaN,PeerChannel(channel_id=1433731512),NaN,Сколько всякой херни понаписали,text,NaN,...,Сколько всякой херни понаписали,Counter(),Сколько всякой херни понаписали,[],Сколько всякой херни понаписали,[],Сколько всякой херни понаписали,[],set(),Сколько всякой херни понаписали
640616,745859,560.0,2019-12-11 16:04:06+00:00,14966.0,NaN,PeerChannel(channel_id=1079904535),NaN,Вы наверняка слышали про профессорку Веру Афан...,text,NaN,...,Вы наверняка слышали про профессорку Веру Афан...,Counter(),Вы наверняка слышали про профессорку Веру Афан...,[],Вы наверняка слышали про профессорку Веру Афан...,"['veraafanasyeva', 'veraafanasyeva']",Вы наверняка слышали про профессорку Веру Афан...,[],set(),Вы наверняка слышали про профессорку Веру Афан...
2067175,2538776,492.0,2017-08-15 15:04:38+00:00,4005.0,NaN,PeerChannel(channel_id=1100158992),NaN,📹 Дерево убило 11 человек на празднике Богомат...,video,37.0,...,📹 Дерево убило 11 человек на празднике Богомат...,Counter({'📹': 1}),Дерево убило 11 человек на празднике Богомат...,[],Дерево убило 11 человек на празднике Богомат...,[],Дерево убило 11 человек на празднике Богоматер...,[],set(),Дерево убило 11 человек на празднике Богоматер...


In [6]:
df_channels = df_channels.drop(columns=['views',
                                        'reactions',
                                        'to_id',
                                        'fwd_from',
                                        'type',
                                        'duration',
                                        'frw_from_title',
                                        'frw_from_name',
                                        'msg_entity',
                                        'is_post',
                                        'message_no_breaks',
                                        'emojis',
                                        'message_no_emoji',
                                        'url_list',
                                        'message_no_urls',
                                        'mentions',
                                        'message_no_mentions',
                                        'quotes',
                                        'hashtags',
                                        'Unnamed: 0'])

In [9]:
%%time
df_channels.columns

CPU times: user 8 µs, sys: 51 µs, total: 59 µs
Wall time: 168 µs


Index(['id', 'date', 'message', 'channel_name', 'lang', 'message_no_hashtags'], dtype='object')

In [8]:
# df_channels.to_csv('/Users/katerynaburovova/PycharmProjects/dehumanization/datasets/df_channels_only_messages.csv')

## Remove everything except letters and punctuation

In [10]:
from string import punctuation

punctuation_minimal = "!(),-.:;?%"

cyrillic_letters = u"абвгдеёжзийклмнопрстуфхцчшщъыьэюяАБВГДЕЁЖЗИЙКЛМНОПРСТУФХЦЧШЩЪЫЬЭЮЯ"
latin_letters_numbers = "1234567890abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ "
allowed_symbols = cyrillic_letters+latin_letters_numbers+punctuation_minimal


In [11]:
allowed_symbols

'абвгдеёжзийклмнопрстуфхцчшщъыьэюяАБВГДЕЁЖЗИЙКЛМНОПРСТУФХЦЧШЩЪЫЬЭЮЯ1234567890abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ !(),-.:;?%'

In [12]:
def clean_text(string, allowed_symbols=allowed_symbols):
    getVals = list(filter(lambda x: x in allowed_symbols, string))
    result = "".join(getVals)
    result = re.sub(' +', ' ', result)

    return result

In [13]:
df_channels['message_no_hashtags'] = df_channels['message_no_hashtags'].astype(str)

In [ ]:
%%time
df_channels['message_clean'] = df_channels['message_no_hashtags'].apply(lambda x: clean_text(x))

In [ ]:
print(f' We have {len(df_channels[df_channels["message_clean"]==""])} ({100*len(df_channels[df_channels["message_clean"]==""])/len(df_channels):2f}%) empty clean messages')

In [ ]:
print(f' We have {len(df_channels[df_channels["message_clean"]!=""])} ({100*len(df_channels[df_channels["message_clean"]!=""])/len(df_channels):2f}%) meaningful clean messages')

In [ ]:
only_letters = cyrillic_letters+latin_letters_numbers

In [ ]:
%%time
df_channels['message_words_only_lower'] = df_channels['message_clean'].apply(lambda x: clean_text(x, allowed_symbols=only_letters).lower())

In [ ]:
df_channels['message_words_only_lower'].sample(10).iloc[0]

### The latest common date

In [ ]:
# Load the data into a pandas dataframe
df = pd.read_csv('telegram_posts.csv')

# Convert the date column to a datetime format
df['date'] = pd.to_datetime(df['date'])

# Group the dataframe by the channel column and find the minimum and maximum dates
grouped = df.groupby('channel')['date'].agg(['min', 'max'])

# Find the latest common post date
latest_common_post_date = grouped['max'].min()

print('The latest common post date is:', latest_common_post_date)


# Approaches

## Spacy RU model

In [ ]:
import spacy
from spacy.lang.ru.examples import sentences

In [ ]:
#python3 -m spacy download ru_core_news_md

In [ ]:
# spacy.util.set_data_path("/Users/katerynaburovova/PycharmProjects/dehumanization/lib/python3.10/site-packages/ru_core_news_md")

In [ ]:
nlp = spacy.load('ru_core_news_md')

In [81]:
def tokenize_spacy(text):
    doc = nlp(text)
    result = []
    for token in doc:
        result.append([token.text, token.pos_, token.dep_, token.lemma_, token.morph])
    return result

In [82]:
tokenize_spacy(sentences[0])

[['Apple',
  'PROPN',
  'nsubj',
  'apple',
  Animacy=Inan|Case=Nom|Gender=Masc|Number=Sing],
 ['рассматривает',
  'VERB',
  'ROOT',
  'рассматривать',
  Aspect=Imp|Mood=Ind|Number=Sing|Person=Third|Tense=Pres|VerbForm=Fin|Voice=Act],
 ['возможность',
  'NOUN',
  'obj',
  'возможность',
  Animacy=Inan|Case=Acc|Gender=Fem|Number=Sing],
 ['покупки',
  'NOUN',
  'nmod',
  'покупка',
  Animacy=Inan|Case=Gen|Gender=Fem|Number=Sing],
 ['стартапа',
  'NOUN',
  'nmod',
  'стартап',
  Animacy=Inan|Case=Gen|Gender=Masc|Number=Sing],
 ['из', 'ADP', 'case', 'из', ],
 ['Соединённого',
  'ADJ',
  'amod',
  'соединённого',
  Case=Gen|Degree=Pos|Gender=Neut|Number=Sing],
 ['Королевства',
  'PROPN',
  'nmod',
  'королевство',
  Animacy=Inan|Case=Gen|Gender=Neut|Number=Sing],
 ['за', 'ADP', 'case', 'за', ],
 ['$', 'SYM', 'nmod', '$', ],
 ['1', 'NUM', 'appos', '1', ],
 ['млрд',
  'NOUN',
  'nmod',
  'млрд',
  Animacy=Inan|Case=Gen|Gender=Masc|Number=Sing]]

## RAZDEL tokenization

Handles "... - ... " as one word (even for composite ones)

In [226]:
# from razdel import tokenize
# def tokenize_razdel(text):
#     return(list(tokenize(text)))

## spacy_russian_tokenizer

https://github.com/aatimofeev/spacy_russian_tokenizer

In [342]:
# from spacy.lang.ru import Russian
# from spacy_russian_tokenizer import RussianTokenizer, MERGE_PATTERNS
# text = "Не ветер, а какой-то ураган!"
# nlp = Russian()
# doc = nlp(text)
# russian_tokenizer = RussianTokenizer(nlp, MERGE_PATTERNS)
# nlp.add_pipe(russian_tokenizer, name='russian_tokenizer')
# doc = nlp(text)
# print([token.text for token in doc])

TypeError: add() takes exactly 2 positional arguments (339 given)

## lang-uk

[не сабворд](https://github.com/lang-uk/ner-uk/blob/master/doc/tokenization.md)

In [83]:
# from tokenize_uk import tokenize_words
# tokenize_words(text)


## stanza


In [84]:
# import stanza
# stanza.download('ru')


In [85]:
# nlp_ru = stanza.Pipeline('ru', processors='tokenize')

In [86]:
# def get_stanza_tokens(text):
#     doc = nlp_ru(text)
#     word_tokens = [token.text for sent in doc.sentences for token in sent.tokens]
#     # result = [word for word in word_tokens if word not in stop_words_ru]
#     return word_tokens

## Subword (BPE?) BERT

Subword, but weird NER

In [87]:
# from transformers import AutoTokenizer, AutoModel

In [88]:
# tokenizer = AutoTokenizer.from_pretrained("cointegrated/rubert-tiny2")
# model = AutoModel.from_pretrained("cointegrated/rubert-tiny2")

In [308]:
# # pip install transformers sentencepiece
# import torch
# from transformers import AutoTokenizer, AutoModel
# tokenizer = AutoTokenizer.from_pretrained("cointegrated/rubert-tiny2")
# model = AutoModel.from_pretrained("cointegrated/rubert-tiny2")
# # model.cuda()  # uncomment it if you have a GPU
#
# def embed_bert_cls(text, model, tokenizer):
#     t = tokenizer(text, padding=True, truncation=True, return_tensors='pt')
#     with torch.no_grad():
#         model_output = model(**{k: v.to(model.device) for k, v in t.items()})
#     embeddings = model_output.last_hidden_state[:, 0, :]
#     embeddings = torch.nn.functional.normalize(embeddings)
#     return embeddings[0].cpu().numpy()
#
# print(embed_bert_cls('привет мир', model, tokenizer).shape)
# # (312,)

In [348]:
# def tokens_bert_cls(text, model, tokenizer):
#     encoding = tokenizer.encode(text)
#     return tokenizer.convert_ids_to_tokens(encoding)
#
# tokens_bert_cls(test_string, model, tokenizer)
# # (312,)

['[CLS]',
 '[UNK]',
 '11',
 '-',
 'го',
 'полка',
 'гон',
 '##яют',
 'танк',
 '[UNK]',
 '11',
 'полк',
 'Н',
 '##М',
 'ДНР',
 'продолжает',
 'наносить',
 'удары',
 'по',
 'противнику',
 'в',
 'Первомай',
 '##ском',
 '.',
 'На',
 'видео',
 'танковый',
 'батальон',
 'под',
 'командованием',
 'Севера',
 'бьет',
 'по',
 'позициям',
 'ВСУ',
 ',',
 'заодно',
 'заставляя',
 'мета',
 '##ться',
 'и',
 'спешно',
 'удалять',
 '##ся',
 'появи',
 '##вшийся',
 'на',
 'свою',
 'беду',
 'украинский',
 'танк',
 '.',
 '@',
 'war',
 '##gon',
 '##zo',
 '*',
 'наш',
 'проект',
 'существует',
 'на',
 'средства',
 'подписчиков',
 ',',
 'карта',
 'для',
 'помощи',
 '427',
 '##9',
 '380',
 '##6',
 '984',
 '##2',
 '952',
 '##1',
 '[SEP]']

In [349]:
# tokens_bert_cls(text, model, tokenizer)

['[CLS]', 'Не', 'ветер', ',', 'а', 'какой', '-', 'то', 'ураган', '!', '[SEP]']

## Execution time

In [ ]:
# df_test_sample = df_channels.sample(100000)

In [353]:
# %%time
# df_test_sample['spacy_tokens'] = df_test_sample['message'].apply(lambda x: tokenize_spacy(x))

CPU times: user 19.5 s, sys: 1.02 s, total: 20.5 s
Wall time: 21.1 s


In [354]:
# %%time
# df_test_sample['bert_tokens'] = df_test_sample['message'].apply(lambda x: tokens_bert_cls(x, model, tokenizer))

CPU times: user 13 s, sys: 331 ms, total: 13.3 s
Wall time: 13.5 s


In [355]:
# %%time
# df_test_sample['lang_uk_tokens'] = df_test_sample['message'].apply(lambda x: tokenize_words(x))

CPU times: user 1.91 s, sys: 75.3 ms, total: 1.99 s
Wall time: 2.02 s


In [356]:
# %%time
# df_test_sample['razdel_tokens'] = df_test_sample['message'].apply(lambda x: tokenize_razdel(x))

CPU times: user 13.8 s, sys: 2.08 s, total: 15.9 s
Wall time: 16.9 s


In [374]:
# %%time
# df_test_sample['stanza_tokens'] = df_test_sample['message'].sample(100).apply(lambda x: get_stanza_tokens(x))

CPU times: user 2.81 s, sys: 234 ms, total: 3.05 s
Wall time: 3.11 s


In [375]:
# %%time
# df_test_sample['stanza_tokens'] = df_test_sample['message'].apply(lambda x: get_stanza_tokens(x))

CPU times: user 40min 18s, sys: 2min 44s, total: 43min 2s
Wall time: 45min


In [384]:
# from random import randrange
#
# df_channels.message.iloc[randrange(len(df_channels))]

'Отправка по электронной почте 2_5325905093675979275.pdf'

In [401]:
# rnd_text = df_channels.message.iloc[randrange(len(df_channels))]
# print(rnd_text)
#
# tokenize_spacy(rnd_text)

Сообщается о раненых в рядах турецких сил в результате подрыва мины на трассе М4 в провинции Идлиб. Турецкие вертолеты направляются к месту взрыва для эвакуации раненых.


[['Сообщается', 'VERB', 'ROOT', 'сообщаться'],
 ['о', 'ADP', 'case', 'о'],
 ['раненых', 'NOUN', 'obl', 'раненый'],
 ['в', 'ADP', 'case', 'в'],
 ['рядах', 'NOUN', 'nmod', 'ряд'],
 ['турецких', 'ADJ', 'amod', 'турецкий'],
 ['сил', 'NOUN', 'nmod', 'сила'],
 ['в', 'ADP', 'case', 'в'],
 ['результате', 'NOUN', 'obl', 'результат'],
 ['подрыва', 'NOUN', 'nmod', 'подрыв'],
 ['мины', 'NOUN', 'nmod', 'мина'],
 ['на', 'ADP', 'case', 'на'],
 ['трассе', 'NOUN', 'nmod', 'трасса'],
 ['М4', 'PROPN', 'appos', 'м4'],
 ['в', 'ADP', 'case', 'в'],
 ['провинции', 'NOUN', 'nmod', 'провинция'],
 ['Идлиб', 'PROPN', 'appos', 'идлиб'],
 ['.', 'PUNCT', 'punct', '.'],
 ['Турецкие', 'ADJ', 'amod', 'турецкий'],
 ['вертолеты', 'NOUN', 'nsubj', 'вертолёт'],
 ['направляются', 'VERB', 'ROOT', 'направляться'],
 ['к', 'ADP', 'case', 'к'],
 ['месту', 'NOUN', 'obl', 'место'],
 ['взрыва', 'NOUN', 'nmod', 'взрыв'],
 ['для', 'ADP', 'case', 'для'],
 ['эвакуации', 'NOUN', 'nmod', 'эвакуация'],
 ['раненых', 'NOUN', 'nmod', 'ранены

# Tokenization and lemmatization

In [89]:
def tokenize_spacy(text):
    doc = nlp(text)
    result = []
    for token in doc:
        result.append([token.text, token.pos_, token.dep_, token.lemma_, token.morph])
    return result

In [114]:
def get_lemmas(text):
    doc = nlp(text)
    result = []
    for token in doc:
        result.append([token.lemma_])
    return result

In [104]:
df_sample = df_channels.sample(1000)

In [1]:
%%time
df_sample['tokens'] = df_sample['message_words_only_lower'].apply(lambda x: tokenize_spacy(x))

NameError: name 'df_sample' is not defined

In [108]:
df_sample['message_words_only_lower'].iloc[536]

'президент россии владимир путин наградил бориса грызлова орденом за заслуги перед отечеством i степени орден вручен за выдающиеся заслуги в укреплении российской государственности и реализации внешнеполитического курса страны также основанием для награды послужили его значительный вклад в развитие обороннопромышленного комплекса россии и многолетняя добросовестная работа'

In [109]:
df_sample['tokens'].iloc[536]

[['президент',
  'NOUN',
  'nsubj',
  'президент',
  Animacy=Anim|Case=Nom|Gender=Masc|Number=Sing],
 ['россии',
  'PROPN',
  'nmod',
  'россия',
  Animacy=Inan|Case=Gen|Gender=Fem|Number=Sing],
 ['владимир',
  'PROPN',
  'appos',
  'владимир',
  Animacy=Anim|Case=Nom|Gender=Masc|Number=Sing],
 ['путин',
  'PROPN',
  'flat:name',
  'путин',
  Animacy=Anim|Case=Nom|Gender=Masc|Number=Sing],
 ['наградил',
  'VERB',
  'ROOT',
  'наградить',
  Aspect=Perf|Gender=Masc|Mood=Ind|Number=Sing|Tense=Past|VerbForm=Fin|Voice=Act],
 ['бориса',
  'PROPN',
  'obj',
  'борис',
  Animacy=Anim|Case=Acc|Gender=Masc|Number=Sing],
 ['грызлова',
  'PROPN',
  'flat:name',
  'грызлов',
  Animacy=Anim|Case=Acc|Gender=Masc|Number=Sing],
 ['орденом',
  'NOUN',
  'xcomp',
  'орден',
  Animacy=Inan|Case=Ins|Gender=Masc|Number=Sing],
 ['за', 'ADP', 'case', 'за', ],
 ['заслуги',
  'NOUN',
  'obl',
  'заслуга',
  Animacy=Inan|Case=Acc|Gender=Fem|Number=Plur],
 ['перед', 'ADP', 'case', 'перед', ],
 ['отечеством',
  'N

In [115]:
%%time
df_sample['lemmas'] = df_sample['message_words_only_lower'].apply(lambda x: get_lemmas(x))

CPU times: user 16.8 s, sys: 433 ms, total: 17.2 s
Wall time: 19.1 s


In [126]:
df_sample['tokens'].iloc[536]


[['как', 'SCONJ', 'mark', 'как', ],
 ['показало',
  'VERB',
  'parataxis',
  'показать',
  Aspect=Perf|Gender=Neut|Mood=Ind|Number=Sing|Tense=Past|VerbForm=Fin|Voice=Act],
 ['исследование',
  'NOUN',
  'nsubj',
  'исследование',
  Animacy=Inan|Case=Nom|Gender=Neut|Number=Sing],
 ['statista', 'X', 'appos', 'statista', Foreign=Yes],
 ['global', 'X', 'flat:foreign', 'global', Foreign=Yes],
 ['consumer', 'X', 'flat:foreign', 'consumer', Foreign=Yes],
 ['survey', 'X', 'flat:foreign', 'survey', Foreign=Yes],
 ['почти', 'ADV', 'advmod', 'почти', Degree=Pos],
 ['каждый', 'DET', 'det', 'каждый', Case=Nom|Gender=Masc|Number=Sing],
 ['пятый',
  'ADJ',
  'amod',
  'пятый',
  Case=Nom|Degree=Pos|Gender=Masc|Number=Sing],
 ['респондент',
  'NOUN',
  'nsubj',
  'респондент',
  Animacy=Anim|Case=Nom|Gender=Masc|Number=Sing],
 ['из', 'ADP', 'case', 'из', ],
 ['великобритании',
  'PROPN',
  'nmod',
  'великобритания',
  Animacy=Inan|Case=Gen|Gender=Fem|Number=Sing],
 ['18', 'NUM', 'appos', '18', ],
 ['о

In [124]:
df_sample['lemmas'].iloc[45]

[['как'],
 ['показать'],
 ['исследование'],
 ['statista'],
 ['global'],
 ['consumer'],
 ['survey'],
 ['почти'],
 ['каждый'],
 ['пятый'],
 ['респондент'],
 ['из'],
 ['великобритания'],
 ['18'],
 ['обычно'],
 ['страдать'],
 ['от'],
 ['похмелие'],
 ['на'],
 ['следующий'],
 ['утро'],
 ['после'],
 ['приём'],
 ['алкоголь'],
 ['такой'],
 ['образ'],
 ['британец'],
 ['оказаться'],
 ['наиболее'],
 ['подверженный'],
 ['этот'],
 ['недуг'],
 ['среди'],
 ['житель'],
 ['восемь'],
 ['страна'],
 ['участвовать'],
 ['в'],
 ['исследование']]

In [125]:
df_sample['message_words_only_lower'].iloc[56]

'блумберг администрация байдена подталкивает союзников ес к завершению разработки широкого пакета санкций против российских банков и энергетических компаний которые могут быть введены совместно с сша если кремль нападет на украину некоторые потенциальные шаги из списка такие как отключение россии от платежной системы swift считаются крайне проблематичными сказали собеседники агентства ссылаясь на потенциальное нарушение работы мировых рынков энергоносителей и других товаров российского экспорта еще одним предметом беспокойства является перспектива того что путин может прекратить экспорт газа россия обеспечивает около 40 поставок в европу в сухом остатке план что и говорить был превосходный простой и ясный лучше не придумать недостаток у него был только один было совершенно неизвестно как привести его в исполнение'

In [127]:
def get_lemma_vec(text):
    doc = nlp(text)
    result = []
    for token in doc:
        result.append([token.lemma])
    return result

In [141]:
tokenize_spacy('хохляцкого')


[['хохляцкого',
  'ADJ',
  'ROOT',
  'хохляцкого',
  Animacy=Anim|Case=Acc|Degree=Pos|Gender=Masc|Number=Sing]]

# OOV handling

https://spacy.io/usage/processing-pipelines#custom-components-user-hooks

should we add a hook that returns zero vectors for OOV terms?

In [135]:
%%time
df_channels['lemmas'] = df_channels['message_words_only_lower'].apply(lambda x: get_lemmas(x))


KeyboardInterrupt



In [136]:
%%time
df_channels['tokens'] = df_channels['message_words_only_lower'].apply(lambda x: tokenize_spacy(x))

KeyboardInterrupt: 

In [137]:
df_channels

,id,date,views,reactions,to_id,fwd_from,message,type,duration,channel_name,...,message_no_emoji,url_list,message_no_urls,mentions,message_no_mentions,quotes,hashtags,message_no_hashtags,message_clean,message_words_only_lower
0,12347,2022-12-15 16:32:15+00:00,9914.0,MessageReactions(results=[ReactionCount(reacti...,PeerChannel(channel_id=1183570279),NaN,А трансляция тем временем идет. Подписывайтесь...,photo,NaN,mardanaka,...,А трансляция тем временем идет. Подписывайтесь...,[www.youtube.com],А трансляция тем временем идет. Подписывайтесь,[],А трансляция тем временем идет. Подписывайтесь,[],{},А трансляция тем временем идет. Подписывайтесь,А трансляция тем временем идет. Подписывайтесь,а трансляция тем временем идет подписывайтесь
1,12346,2022-12-15 15:00:03+00:00,29207.0,MessageReactions(results=[ReactionCount(reacti...,PeerChannel(channel_id=1183570279),NaN,"В России ещё есть те, кто способен возрождать ...",photo,NaN,mardanaka,...,"В России ещё есть те, кто способен возрождать ...",[],"В России ещё есть те, кто способен возрождать ...",[],"В России ещё есть те, кто способен возрождать ...",[МЭФ. Экономика для людей],{},"В России ещё есть те, кто способен возрождать ...","В России ещё есть те, кто способен возрождать ...",в россии ещё есть те кто способен возрождать н...
2,12345,2022-12-15 14:21:22+00:00,41058.0,MessageReactions(results=[ReactionCount(reacti...,PeerChannel(channel_id=1183570279),NaN,«Катаргейт» набирает обороты - Обвиняемая в ко...,text,NaN,mardanaka,...,«Катаргейт» набирает обороты - Обвиняемая в ко...,[],«Катаргейт» набирает обороты - Обвиняемая в ко...,[],"""Катаргейт"" набирает обороты - Обвиняемая в ко...",[Катаргейт],{},"""Катаргейт"" набирает обороты - Обвиняемая в ко...",Катаргейт набирает обороты - Обвиняемая в корр...,катаргейт набирает обороты обвиняемая в корруп...
3,12344,2022-12-15 13:08:35+00:00,40696.0,MessageReactions(results=[ReactionCount(reacti...,PeerChannel(channel_id=1183570279),"MessageFwdHeader(date=datetime.datetime(2022, ...",🔥В гостях у @Metametrica побывали товарищи из ...,photo,NaN,mardanaka,...,В гостях у @Metametrica побывали товарищи из ...,[],В гостях у @Metametrica побывали товарищи из ...,"[Metametrica, Metametrica]",В гостях у побывали товарищи из украинского п...,[Богдана Хмельницкого],{},В гостях у побывали товарищи из украинского п...,В гостях у побывали товарищи из украинского пр...,в гостях у побывали товарищи из украинского пр...
4,12343,2022-12-15 12:31:23+00:00,51690.0,MessageReactions(results=[ReactionCount(reacti...,PeerChannel(channel_id=1183570279),NaN,Глава Минобороны Грузии назвал грузинских наем...,text,NaN,mardanaka,...,Глава Минобороны Грузии назвал грузинских наем...,[],Глава Минобороны Грузии назвал грузинских наем...,[],Глава Минобороны Грузии назвал грузинских наем...,"[Есть случаи, когда едут воевать за гонорар, в...",{},Глава Минобороны Грузии назвал грузинских наем...,Глава Минобороны Грузии назвал грузинских наем...,глава минобороны грузии назвал грузинских наем...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1607921,7,2019-09-28 10:13:52+00:00,3190.0,NaN,PeerChannel(channel_id=1253974160),NaN,Бот для связи: @obrazbuduschego2_bot Донастрое...,text,NaN,obrazbuduschego2,...,Бот для связи: @obrazbuduschego2_bot Донастрое...,[],Бот для связи: @obrazbuduschego2_bot Донастрое...,"[obrazbuduschego2_bot, protonmail]","Бот для связи: Донастроен, работает, если кто...",[],{},"Бот для связи: Донастроен, работает, если кто...","Бот для связи: Донастроен, работает, если кто-...",бот для связи донастроен работает если ктото н...
1607922,6,2019-09-28 09:35:07+00:00,3239.0,NaN,PeerChannel(channel_id=1253974160),NaN,"Печальная весть о Марке Захарове, человеке, ко...",text,NaN,obrazbuduschego2,...,"Печальная весть о Марке Захарове, человеке, ко...",[],"Печальная весть о Марке Захарове, человеке, ко...",[],"Печальная весть о Марке Захарове, человеке, ко...","[Чай, кофе и другие колониальные товары]",{},"Печальная весть о Мар

In [144]:
df_channels.message_words_only_lower.iloc[33330]

'и посмотрите елену серветтаз она там местами срывает покровы про альбац браудера и так далее и вообще она очень хорошая'

In [ ]:
#pymorphy2
#sparknlp
#на чому тренувався spacy - взяти з цього корпусу словник
#взятисловник
#службові частини мови забрати

In [1]:
import pandas as pd

In [ ]:
%%time
df = pd.read_csv('/Users/katerynaburovova/PycharmProjects/dehumanization/data/merged_dataset/df_channels.csv')

In [3]:
df.columns

Index(['Unnamed: 0', 'id', 'date', 'message', 'channel_name', 'lang',
       'message_no_hashtags', 'date_formatted', 'message_clean',
       'message_final'],
      dtype='object')